# Source Estimation

The SourceEstimate data structure
https://mne.tools/stable/auto_tutorials/inverse/10_stc_class.html

## Import

In [ ]:
import os.path as op

import mne
import os
import mne_bids
import vtk

from mne.forward import read_forward_solution
from mne.minimum_norm import (make_inverse_operator, apply_inverse,
                              write_inverse_operator)

## Directory

In [ ]:
eeg_path = "eeg"
subname = "sub-"+subject
sesname = "ses-"+session
preprocessed_reports = "03_preprocessed"
forward_solution = "04_fwd"
covariance_reports = "05_covariance"
inverse_operator = "06_inv"
source_estimates = "07_stc"

In [ ]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> evoked_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, inverse_operator)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, inverse_operator))

In [ ]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> evoked_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, source_estimates)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, source_estimates))

In [ ]:
preprocessed_report_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+preprocessed_reports)
forward_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+forward_solution)
covariance_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+covariance_reports)
inverse_operator_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+inverse_operator)
source_estimates_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+source_estimates)

## Input Epoch Data

In [ ]:
preprocessed_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_preprocessed_epo.fif'
preprocessed_file = os.path.join(preprocessed_report_path, preprocessed_name)
epochs = mne.read_epochs(preprocessed_file)
epochs

In [ ]:
info = mne.io.read_info(preprocessed_file)

## Input Forward Solution

In [ ]:
fwd_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_fwd.fif'
fwd_file = os.path.join(forward_path, fwd_name)
fwd = mne.read_forward_solution(fwd_file)
fwd

## Input Noise Covariance

In [ ]:
cov_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_cov.fif'
cov_file = os.path.join(covariance_path, cov_name)
noise_cov = mne.read_cov(cov_file)
noise_cov

## Inverse Operator

In [ ]:
inverse_operator = make_inverse_operator(info, fwd, noise_cov,
                                         loose=0.2, depth=0.8)

### Write Inverse Operator

In [ ]:
inv_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'-inv.fif'

inv_path = os.path.join(inverse_operator_path, inv_title)
mne.minimum_norm.write_inverse_operator(inv_path, inverse_operator, overwrite=True)

## Calculate Source Estimate

The Evoked data structure: evoked/averaged data
https://mne.tools/stable/auto_tutorials/evoked/10_evoked_overview.html

Source localization with MNE, dSPM, sLORETA, and eLORETA
https://mne.tools/stable/auto_tutorials/inverse/30_mne_dspm_loreta.html

Computing various MNE solutions
https://mne.tools/stable/auto_tutorials/inverse/40_mne_fixed_free.html#

In [ ]:
print('######################## Computing source estimate. ########################')

In [ ]:
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2

stc = apply_inverse(epochs.average(), inverse_operator, lambda2,
                    method=method, pick_ori=None)

### Show Source Estimate

## Report Source Estimate

In [ ]:
stc_report_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_stc_report.html'
stc_report_path = os.path.join(source_estimates_path, stc_report_title)


stc_report = mne.Report(title='Source Estimate')
stc_report.add_stc(
    stc=stc, subject='fsaverage',
    title='Source Estimate', n_time_points=6  # few for speed
)
stc_report.save(stc_report_path, overwrite=True)

## Save Source Estimate

In [ ]:
stc_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_source_estimate.stc'

stc_path = os.path.join(source_estimates_path, stc_title)
stc.save(stc_path, overwrite=True)